In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import json

url = "https://www.baraasallout.com/test.html"
response = requests.get(url)

if response.status_code == 200:
    print("================")
    print("Good to go")
    print("================")

    page_content = response.text
    soup = BeautifulSoup(page_content, "html.parser")
    book_cards = soup.find_all('div') 

    title = soup.find("title").text
    print("Title:", title)
    print("")

    # Lists to store data
    headings = []
    paragraphs = []
    lists = []

    table_data = []
    Table_headers = []

    book_data = []

    input_feild_data = []

    hyperlinks = []
    video_links = []
    featured_products = []
    
    table = soup.find('table')
    form = soup.find('form')
    links = soup.find_all('a')
    videos = soup.find_all('iframe')
    featured_product = soup.find_all('div', class_='featured-product')


    # HEADER: Extract headings (h1, h2, ..., h6)
    for headers in soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6']):
        headings.append(headers.text.strip())

    # PARAGRAPHS: Extract paragraphs (<p>)
    for parag in soup.find_all('p'):
        paragraphs.append(parag.text.strip())

    # LISTS: Extract list items (<li>)
    for items in soup.find_all('li'):
        lists.append(items.text.strip())

    Table_header_row = table.find_all('th')
    for header in Table_header_row:
        Table_headers.append(header.text.strip())

    # Add headers to the table_data list
    table_data.append(Table_headers)

    # Extract all rows from the table (excluding the header)
    rows = table.find_all('tr')[1:]  # Skip the first row which is the header
    for row in rows:
        columns = row.find_all('td')
        column_data = [column.text.strip() for column in columns]
        table_data.append(column_data)


    for card in book_cards:
        # Extract the book title, price, stock availability, and button text
        book_title = card.find("strong")
        price = card.find('p', style="color: green;")  # Price is within a <p> tag with this inline style
        stock = card.find_all('p', style="color: green;")[1] if len(card.find_all('p', style="color: green;")) > 1 else None
        button_text = card.find('button')

        # Only proceed if all necessary elements are found
        if book_title and price and stock and button_text:
            book_title_text = book_title.text.strip()
            book_price_text = price.text.strip()
            book_stock_text = stock.text.strip() if stock else "Stock info not available"
            button_text = button_text.text.strip()

            book_data.append({
                "Book Title": book_title_text,
                "Price": book_price_text,
                "Stock": book_stock_text,
                "Button Text": button_text
            })

           
        
        form = soup.find('form')
    
    if form:
        inputs = form.find_all('input')
        print("Input Fields:")
        for input in inputs:
            field_name = input.get('name')
            field_type = input.get('type')
            placeholder = input.get('placeholder')


        # Append the data to the list
            input_feild_data.append({
                "Name": field_name,
                "Type": field_type,
             "Placeholder": placeholder
            })


    for link in links:

        link_text = link.text.strip()
        href = link.get('href')

        hyperlinks.append({
            "text": link_text,
            "href" : href
        })

    for video in videos:
        src = video.get('src')  # Default to 'N/A' if src is missing
        video_links.append({
        "Iframe Source": src
    })



    product_cards = soup.find_all('div', class_='product-card')

    for product in product_cards:

        
        product_id = product.get('data-id')
        
        product_name = product.find('p', class_='name').text.strip() 
        
        product_price = product.find('p', class_='price', style='display: none;')
        product_price = product_price.text.strip()

        product_colors = product.find('p', class_='colors').text.strip()

        print(f"{product_id} , {product_name} , {product_price} , {product_colors}")





    



    # Write data to CSV
    with open("Extract_Text_Data.csv", 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        
        writer.writerow(["Title", title])
        writer.writerow([])


        for heading in headings:
            writer.writerow(["Header", heading])
        writer.writerow([])

        for para in paragraphs:
            writer.writerow(["Paragraph", para])
        writer.writerow([])

        for items in lists:
            writer.writerow(["LIst", items])
        writer.writerow([])

        writer.writerows(table_data)
        writer.writerow([])

    print("Data has been saved to 'Extract_Text_Data.csv'.")

    with open("Product_Information.json", "w", encoding="utf-8") as json_file:
        json.dump(book_data, json_file, indent=4, ensure_ascii=False)

    print("Data has been saved to 'Product_Information.json'.")

    with open("Input_feild_data.json", "w", encoding="utf-8") as json_file:
        json.dump(input_feild_data, json_file, indent=4, ensure_ascii=False)
    print("Data has been saved to 'Input_feild_data.json'.")

   

    links_data = {
    "Hyperlinks": hyperlinks,
    "VideoLinks": video_links
    }

# Save the combined data to a JSON file
    with open("Links.json", "w", encoding="utf-8") as json_file:
        json.dump(links_data, json_file, indent=4, ensure_ascii=False)

    print("Data has been saved to 'Links.json'.")



    # Optionally, save the data to a JSON file
    with open("featured_products.json", "w", encoding="utf-8") as json_file:
        json.dump(featured_products, json_file, indent=4, ensure_ascii=False)

    print("Data has been saved to 'featured_products.json'.")


else:
    print("Error: Failed to fetch the page")
